## Columns of interest


http://insideairbnb.com/get-the-data.html
Boston data 17November2018

### calendar.csv

* available
* price

### listings.csv

(?) can we find most frequent words in these fields?
* summary
* space
* descriptions
* neighborhood_overview
* notes
* transit

Others
* host_since
* host_location
* host_about (Can we extract common texts in this field")
* host_response_time
* host_response_rate
* host_is_superhost
* host_neighbourhood
* host_listings_count
* host_verifications
* host_identity_verified
* street
* neighbourhood
* neighbourhood_cleansed
* neighbourhood_group_cleansed
* city
* state
* zipcode
* is_location_exact
* property_type (Might need to be cleaned since the same property can have multiple listing with this info)
* room_type
* accommodates
* bathrooms
* bedrooms
* beds
* bed_type
* amenities (Need to clean this)
* price
* weekly_price
* monthly_price
* security_deposit
* cleaning_fee
* guests_included
* extra_people
* minimum_nights
* maximum_nights
* availability_30
* availability_60
* availability_90
* availability_365
* number_of_reviews
* first_review
* last_review
* review_scores_rating
* review_scores_accuracy
* review_scores_cleanliness
* review_scores_checkin
* review_scores_communication
* review_scores_location
* review_scores_value
* instant_bookable
* cancellation_policy
* reviews_per_month

Prepare data:

* Gather necessary data to answer your questions
* Handle categorical and missing data
* Provide insight into the methods you chose and why you chose them

Analyze, Model, and Visualize

* Provide a clear connection between your business questions and how the data answers them.

Questions:
* What artifacts in listing that help with more booking/reviews?
* Great boston areas -> average price for each room type